# Move files exported by Equivital Qiosk

Check and clear 

Plot and review measurements from equivital sensors as output by the Equivital Manager app.
(Every program produces different formated files and it's a pain.)

In [10]:
import sys
import os
import shutil
import time
import datetime as dt
import math
import numpy as np 
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# import heartpy as hp

from scipy.signal import butter,filtfilt
from scipy import interpolate
from scipy.interpolate import interp1d

## Definitions for data extraction

In [11]:
%load_ext autoreload
%autoreload 1
%aimport qex

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
%reload_ext autoreload

In [13]:
seperator = str('\\')
seperator

'\\'

# Scan for DATA files

All recordinds exported by Qiosk produce four CSV files and one SEM file. One of the CSVs is a DATA file that contains metadata and some essential signal quality checks on 15 s intervals.

We can ID instances of signal recordings from statistics taken out of the DATA file, so that gets extracted first

In [21]:
# location of the files when exported by QIOSK
path = "C:\\Users\\Public\\Documents\\Equivital\\Equivital Manager Wizard\\"#Extracted SEM Data\\" #"../Equivital/Equivital Manager Wizard/Extracted SEM Data/"
#os.listdir(path)

df_datafiles = qex.qiosk_recordings(path,'Present',seperator)
df_datafiles

,Signal,DevName,ID,Date,Session,FileName,FileType,FileSize,RecStart,RecEnd,...,SubjectNames,SENSOR ID,SUBJECT ID,SUBJECT AGE,HR(BPM),HRC(%),BELT OFF,LEAD OFF,MOTION,BODY POSITION
0,DATA,Bluetooth02,5022285,230216,0,DATA-Bluetooth02-5022285-23021600.CSV,csv,510330,2023-02-16 07:10:40+00:00,2023-02-17 07:43:10+00:00,...,[Bluetooth02],5022285.0,Bluetooth02,NaN,26.0,100.0,1.0,1.0,Stationary,Side
1,DATA,LS410,5022154,230216,0,DATA-LS410-5022154-23021600.CSV,csv,567466,2023-02-16 07:10:44+00:00,2023-02-17 12:13:44+00:00,...,[LS410],5022154.0,LS410,NaN,2.0,99.0,0.0,1.0,Stationary,Prone
2,DATA,VN102,3420426,230217,0,DATA-VN102-3420426-23021700.CSV,csv,96165,2023-02-17 07:09:37+00:00,2023-02-17 12:00:07+00:00,...,[VN102],3420426.0,VN102,NaN,0.0,97.0,0.0,0.0,Stationary,Upright
3,DATA,VN205,5022060,230217,0,DATA-VN205-5022060-23021700.CSV,csv,97835,2023-02-17 07:09:43+00:00,2023-02-17 12:04:58+00:00,...,[VN205],5022060.0,VN205,NaN,55.0,99.0,0.0,0.0,Stationary,Upright
4,DATA,VN104,3420456,230217,0,DATA-VN104-3420456-23021700.CSV,csv,96190,2023-02-17 07:11:02+00:00,2023-02-17 12:00:18+00:00,...,[VN104],3420456.0,VN104,NaN,0.0,98.0,0.0,0.0,Stationary,Upright
5,DATA,VA305,5022117,230217,0,DATA-VA305-5022117-23021700.CSV,csv,100057,2023-02-17 07:11:09+00:00,2023-02-17 12:13:39+00:00,...,[VA305],5022117.0,VA305,NaN,63.0,98.0,0.0,0.0,Stationary,Upright
6,DATA,PC707,5022256,230217,101,DATA-PC707-5022256-23021700_0.CSV,csv,92186,2023-02-17 07:11:10+00:00,2023-02-17 11:48:55+00:00,...,[PC707],5022256.0,PC707,NaN,0.0,98.0,0.0,0.0,Stationary,Upright
7,DATA,WW506,5022166,230217,0,DATA-WW506-5022166-23021700.CSV,csv,99523,2023-02-17 07:11:12+00:00,2023-02-17 12:11:42+00:00,...,[WW506],5022166.0,WW506,NaN,92.0,97.0,0.0,0.0,Stationary,Upright
8,DATA,WW501,5022155,230217,0,DATA-WW501-5022155-23021700.CSV,csv,99151,2023-02-17 07:11:12+00:00,2023-02-17 12:11:42+00:00,...,[WW501],5022155.0,WW501,NaN,78.0,98.0,0.0,1.0,Stationary,Prone
9,DATA,LS403,5022141,230217,0,DATA-LS403-5022141-23021700.CSV,csv,100051,2023-02-17 07:11:15+00:00,2023-02-17 12:13:45+00:00,...,[LS403],5022141.0,LS403,NaN,84.0,98.0,0.0,0.0,Stationary,Upright


In [22]:
print('Number of DATA files: '+ str(len(df_datafiles)))
# key categories to structure of Qiosk output DATA files
print(df_datafiles['ID'].unique())
print(df_datafiles['DevName'].unique())
print(df_datafiles['Date'].unique())

Number of DATA files: 56
[5022285 5022154 3420426 5022060 3420456 5022117 5022256 5022166 5022155
 5022141 5022139 5022240 5022076 5022073 5022002 5022077 3420415 5022006
 3420427 3420755 5022009 5022131 5022138 5022151 5023926 5022143 5022149
 5022142 5022051 5022050 5022058 5022057 5022062 5022098 5022163 5022173
 5022161 5022156 5022202 5022164 5022127 5022126 5022104 5022100 5022101
 5022110 5022204 5022193 5022203 5022198 5022184 5022179 5022244 5022245
 5022241 5022238]
['Bluetooth02' 'LS410' 'VN102' 'VN205' 'VN104' 'VA305' 'PC707' 'WW506'
 'WW501' 'LS403' 'LS402' 'PC702' 'VN208' 'VN207' 'VN106' 'VN209' 'VN101'
 'VN107' 'VN103' 'VN105' 'VN108' 'VA308' 'LS401' 'LS408' 'LS406' 'LS405'
 'LS407' 'LS404' 'VN202' 'VN201' 'VN204' 'VN203' 'VN206' 'VN210' 'WW504'
 'WW507' 'WW503' 'WW502' 'BR605' 'WW505' 'VA307' 'VA306' 'VA303' 'VA301'
 'VA302' 'VA304' 'BR607' 'BR603' 'BR606' 'BR604' 'BR602' 'BR601' 'PC704'
 'PC705' 'PC703' 'PC701']
[230216 230217]


## Define set of recordings to move
First specify a subset by either device number (singular) or day of recording event.

Next this subset can be refined by restricting recordings to a specific time of day: to have started before a particular time (when you know the recording was happening) and/or to have ended after a specific time (when you know the recording was happening.

All times are in UTC, which is an hour earlier than Central European Time, 2 hours earlier than Central European Summer Time.


In [16]:
# all the recordings that are in the directiory
s_files = df_datafiles

In [66]:
# DATA files from one device, by device ID numbr
devN = 3420456
s_files=df_datafiles.loc[df_datafiles['ID'] == devN,:] 

In [23]:
# DATA files from one device, by device ID numbr
devN = 'Bluetooth02'
s_files=df_datafiles.loc[df_datafiles['DevName'] == devN,:] 

In [108]:
# DATA files from recordings on a particular day, YYMMDD
recDate = 230214
s_files=df_datafiles.loc[df_datafiles['Date'] == recDate,:]


In [9]:
# DATA files from recordings that lasted longer than N seconds
minDuration = 600  
s_files=df_datafiles.loc[df_datafiles['Duration'] > minDuration,:]


In [17]:
# DATA files from recordings that are smaller than N Bytes
maxfileSize = 500 
s_files=df_datafiles.loc[df_datafiles['FileSize'] < maxfileSize,:]


In [91]:
# DATA files from recordings that are smaller than N Bytes
maxdurSize = 50 
s_files=s_files.loc[s_files['Duration'] < maxdurSize,:]


Additionally select according to start and stop recording times.

In [82]:
# recordings started before a certain time 
t = pd.to_datetime('2023-02-14 12:02:00.00+0000')# V['DateTime'].iloc[int(len(V)/2)]   2023-01-18 13:30:09 2023-01-18 13:43:27
s_files=s_files.loc[s_files['RecStart']< t,:] 

In [68]:
# recordings ended after a certain time
t = pd.to_datetime('2023-01-28 00:00:00.00+0000')# V['DateTime'].iloc[int(len(V)/2)]   2023-01-18 13:30:09 2023-01-18 13:43:27
s_files=s_files.loc[s_files['RecEnd']> t,:] 

In [24]:
# show subset of files
s_files

,Signal,DevName,ID,Date,Session,FileName,FileType,FileSize,RecStart,RecEnd,...,SubjectNames,SENSOR ID,SUBJECT ID,SUBJECT AGE,HR(BPM),HRC(%),BELT OFF,LEAD OFF,MOTION,BODY POSITION
0,DATA,Bluetooth02,5022285,230216,0,DATA-Bluetooth02-5022285-23021600.CSV,csv,510330,2023-02-16 07:10:40+00:00,2023-02-17 07:43:10+00:00,...,[Bluetooth02],5022285.0,Bluetooth02,NaN,26.0,100.0,1.0,1.0,Stationary,Side


These recordings can be checked in View_Equivital_Qiosk_Export.

Now we move them

# Move selected files to suitable project folder

In [70]:
projectsFolder = 'C:\\Users\\fourMs lab\\Documents\\Equivital\\ProjectData\\' # where qiosk files are moved to after export
print(path) # where the qiosk files are initially saved by the Equivital Qiosk program

C:\Users\Public\Documents\Equivital\Equivital Manager Wizard\


In [71]:
# set project folder name
projectName = 'AlexanderStandStill'
#projectName = 'BatteryTests'
#projectName = 'SoloRecordings'
#projectName = 'Concert230128'
#projectName = 'LauraTestRecording'
projectPath = projectsFolder + projectName + '\\'

### Create folders for project to move files to
Check target location for data files to keep and to remove

In [72]:
# check if the folder exists for project and set up locations
if not os.path.isdir(projectsFolder + projectName):
    os.mkdir(projectsFolder + projectName)
    
if not os.path.isdir(projectsFolder + projectName + '\\' + 'SEM'):
    os.mkdir(projectsFolder + projectName + '\\' + 'SEM')
if not os.path.isdir(projectsFolder + projectName + '\\' + 'CSV'):
    os.mkdir(projectsFolder + projectName + '\\' + 'CSV')

### Move CSV and SEM files to project folder
Move them to project folder and generate their own present recordings file.

In [73]:
for i,row in s_files.iterrows():
    dataFile = row['FullLoc']
    matched = qex.matched_files(dataFile,path) # outputs locations of csv and sem files
    
    for fi in matched:
        fileName = fi.split('\\')[-1]
        devName = row['DevName']
        if fileName.lower().endswith('csv'):
            if not fileName.lower().endswith('Recordings.csv'):
                out_f = projectPath + 'CSV\\' + fileName
                os.rename(fi,out_f)
        if fileName.lower().endswith('sem'):
            if not os.path.isdir(projectsFolder + projectName + '\\SEM\\' + devName):
                os.mkdir(projectsFolder + projectName  + '\\SEM\\' + devName)
            out_f = projectPath + 'SEM\\' + devName  + '\\' + fileName
            os.rename(fi,out_f)
            
dfiles = qex.qiosk_recordings(projectPath,projectName)
dfiles

,Signal,DevName,ID,Date,Session,FileName,FileType,FileSize,RecStart,RecEnd,...,SubjectNames,SENSOR ID,SUBJECT ID,SUBJECT AGE,HR(BPM),HRC(%),BELT OFF,LEAD OFF,MOTION,BODY POSITION
0,DATA,Pilot_1,3420755,230131,0,DATA-Pilot_1-3420755-23013100.CSV,csv,12267,2023-01-31 14:38:04+00:00,2023-01-31 15:15:49+00:00,...,[Pilot_1],3420755.0,Pilot_1,NaN,79,95.0,0.0,1.0,Stationary,Upright
1,DATA,Pilot_1,3420755,230201,101,DATA-Pilot_1-3420755-23020100_0.CSV,csv,5366,2023-02-01 12:01:29+00:00,2023-02-01 12:17:44+00:00,...,[Pilot_1],3420755.0,Pilot_1,NaN,84,100.0,0.0,0.0,Stationary,Upright
2,DATA,Pilot_2,3420456,230203,0,DATA-Pilot_2-3420456-23020300.CSV,csv,8532,2023-02-03 11:40:06+00:00,2023-02-03 12:06:36+00:00,...,[Pilot_2],3420456.0,Pilot_2,NaN,0,99.0,0.0,0.0,Stationary,Upright


# Clear Recordings to ToBeDeleted

In [25]:
# if these are files to be discarded store the DATA file indexes to discardRecs
discardRecs = s_files.index
projTag = 'ToBeDiscarded'
discardPath = 'C:\\Users\\fourMs lab\\Documents\\Equivital\\ProjectData\\ToBeDeleted\\'

for i,row in s_files.iterrows():
    dataFile = row['FullLoc']
    matched = qex.matched_files(dataFile,path,seperator) # outputs locations of csv and sem files
    
    for f in matched:
        fileName = f.split('\\')[-1]
        devName = row['DevName']
        if f.lower().endswith('csv'):
            out_f = discardPath + 'CSV\\' + fileName
            shutil.move(f,out_f)
        if f.lower().endswith('sem'):
            if not os.path.isdir(discardPath + 'SEM\\' + devName):
                os.mkdir(discardPath + 'SEM\\' + devName)
            out_f = discardPath + 'SEM\\' + devName + '\\' + fileName
            shutil.move(f,out_f)
            


In [26]:
dfiles = qex.qiosk_recordings(discardPath,projTag,seperator)
print(len(dfiles))

836


# if Files need to be moved between project folders

Set the origin project name and target project name, collect the existing files at that location, chose the subset of recordings to move or copy, move or copy them, generate recording list in each location

In [40]:
# incase files need to be shifted from a project folder
projectName = 'SoloRecordings'
projectPath = projectsFolder + projectName + '\\'
projectPath1 = projectPath
projectName1 = projectName

s_files = qex.qiosk_recordings(projectPath1,projectName1)
s_files 

,Signal,DevName,ID,Date,Session,FileName,FileType,FileSize,RecStart,RecEnd,...,SubjectNames,SENSOR ID,SUBJECT ID,SUBJECT AGE,HR(BPM),HRC(%),BELT OFF,LEAD OFF,MOTION,BODY POSITION
0,DATA,Test1,3420415,230120,7,DATA-Test1-3420415-23012007.CSV,csv,858075,2023-01-20 22:00:08+00:00,2023-01-21 13:31:13+00:00,...,[Test1],3420415.0,Test1,NaN,61,99,0,0,MovingSlowly,Side
1,DATA,Pilot_1,3420755,230201,102,DATA-Pilot_1-3420755-23020100_1.CSV,csv,62494,2023-02-01 17:14:56+00:00,2023-02-01 20:30:11+00:00,...,[Pilot_1],3420755.0,Pilot_1,NaN,78,98,0,0,Stationary,Upright
2,DATA,Bluetooth_Tester_1,3420415,230203,0,DATA-Bluetooth_Tester_1-3420415-23020300.CSV,csv,196,2023-02-03 10:10:03+00:00,2023-02-03 10:10:03+00:00,...,[Bluetooth_Tester_1],3420415.0,Bluetooth_Tester_1,NaN,30,97,1,1,Stationary,Side


In [41]:
# get subset of s_files as you see fit
s_files = s_files.loc[2:2,:].copy()
s_files

,Signal,DevName,ID,Date,Session,FileName,FileType,FileSize,RecStart,RecEnd,...,SubjectNames,SENSOR ID,SUBJECT ID,SUBJECT AGE,HR(BPM),HRC(%),BELT OFF,LEAD OFF,MOTION,BODY POSITION
2,DATA,Bluetooth_Tester_1,3420415,230203,0,DATA-Bluetooth_Tester_1-3420415-23020300.CSV,csv,196,2023-02-03 10:10:03+00:00,2023-02-03 10:10:03+00:00,...,[Bluetooth_Tester_1],3420415.0,Bluetooth_Tester_1,NaN,30,97,1,1,Stationary,Side


In [42]:
# project these files should be moved to
projectName = 'ToBeDeleted'
# check if the folder exists for project and set up locations
if not os.path.isdir(projectsFolder + projectName):
    os.mkdir(projectsFolder + projectName)
    
if not os.path.isdir(projectsFolder + projectName + '\\' + 'SEM'):
    os.mkdir(projectsFolder + projectName + '\\' + 'SEM')
if not os.path.isdir(projectsFolder + projectName + '\\' + 'CSV'):
    os.mkdir(projectsFolder + projectName + '\\' + 'CSV')
    
projectPath = projectsFolder + projectName + '\\'
projectPath2 = projectPath
projectName2 = projectName

In [43]:
print(row['FullLoc'])
matched = qex.matched_files(dataFile,projectPath1)

C:\Users\fourMs lab\Documents\Equivital\ProjectData\SoloRecordings\CSV\DATA-Bluetooth_Tester_1-3420415-23020300.CSV


In [54]:
# move s_files
for i,row in s_files.iterrows():
    dataFile = row['FullLoc']
    matched = qex.matched_files(dataFile,projectPath1) # outputs locations of csv and sem files
    
    for fi in matched:
        fileName = fi.split('\\')[-1]
        devName = row['DevName']
        if fileName.lower().endswith('csv'):
            if not fileName.lower().endswith('Recordings.csv'):
                out_f = projectPath2 + 'CSV\\' + fileName
                shutil.move(fi,out_f)
        if fileName.lower().endswith('sem'):
            if not os.path.isdir(projectPath2 + 'SEM\\' + devName):
                os.mkdir(projectPath2 + 'SEM\\' + devName)
            out_f = projectPath2 + 'SEM\\' + devName  + '\\' + fileName
            shutil.move(fi,out_f)

In [49]:
# COPY s_files
for i,row in s_files.iterrows():
    dataFile = row['FullLoc']
    matched = qex.matched_files(dataFile,projectPath1) # outputs locations of csv and sem files
    
    for fi in matched:
        fileName = fi.split('\\')[-1]
        devName = row['DevName']
        if fileName.lower().endswith('csv'):
            if not fileName.lower().endswith('Recordings.csv'):
                out_f = projectPath2 + 'CSV\\' + fileName
                #os.system('cp ' + fi + ' ' + out_f)
                shutil.copy2(fi,out_f) # complete target filename given

        if fileName.lower().endswith('sem'):
            if not os.path.isdir(projectPath2 + 'SEM\\' + devName):
                os.mkdir(projectPath2 + 'SEM\\' + devName)
            out_f = projectPath2 + 'SEM\\' + devName  + '\\' + fileName
            shutil.copy2(fi,out_f) # complete target filename given


In [50]:
fi

'C:\\Users\\fourMs lab\\Documents\\Equivital\\ProjectData\\SoloRecordings\\SEM\\Bluetooth_Tester_1\\23020300.SEM'

In [51]:
print(projectPath2)
out_f

C:\Users\fourMs lab\Documents\Equivital\ProjectData\ToBeDeleted\


'C:\\Users\\fourMs lab\\Documents\\Equivital\\ProjectData\\ToBeDeleted\\SEM\\Bluetooth_Tester_1\\23020300.SEM'

In [55]:
dfiles = qex.qiosk_recordings(projectPath1,projectName1)
print(projectPath1+projectName1)
print(dfiles)
dfiles = qex.qiosk_recordings(projectPath2,projectName2)
print(projectPath2+projectName2)
print(dfiles)

C:\Users\fourMs lab\Documents\Equivital\ProjectData\SoloRecordings\SoloRecordings
  Signal  DevName       ID    Date  Session  \
0   DATA    Test1  3420415  230120        7   
1   DATA  Pilot_1  3420755  230201      102   

                              FileName FileType  FileSize  \
0      DATA-Test1-3420415-23012007.CSV      csv    858075   
1  DATA-Pilot_1-3420755-23020100_1.CSV      csv     62494   

                   RecStart                    RecEnd  ...  SubjectNames  \
0 2023-01-20 22:00:08+00:00 2023-01-21 13:31:13+00:00  ...       [Test1]   
1 2023-02-01 17:14:56+00:00 2023-02-01 20:30:11+00:00  ...     [Pilot_1]   

   SENSOR ID  SUBJECT ID  SUBJECT AGE HR(BPM) HRC(%)  BELT OFF LEAD OFF  \
0  3420415.0       Test1          NaN      61     99         0        0   
1  3420755.0     Pilot_1          NaN      78     98         0        0   

         MOTION  BODY POSITION  
0  MovingSlowly           Side  
1    Stationary        Upright  

[2 rows x 25 columns]
C:\Users\fourMs